- 전처리 사항
1. 종별코드명에서 '약국' 삭제하기
2. 진료과목정보 암호화요양기호, 진료과목코드명으로 groupby해서 merge하기
3. 일반입원실일반, 일반입원실상급 합쳐서 일반입원실병상수 칼럼으로 만들기
4. 안쓰는 칼럼 삭제
5. 컬럼에 _x 이런거 없애기
6. 일반입원실병상수 위치 옮기기
7. '개설일자' 타입 int에서 object로

## 2024년

In [157]:
import pandas as pd

In [158]:
DATA_PATH = '/Users/hong/Desktop/practice_code/mid_project/data/전국병의원및약국현황/전국 병의원 및 약국 현황 2024.9/'

file_lst = ['1.병원정보서비스 2024.9', '3.의료기관별상세정보서비스_01_시설정보 2024.9',
            '5.의료기관별상세정보서비스_03_진료과목정보 2024.9']

In [159]:
df = pd.read_excel(f'{DATA_PATH}{file_lst[0]}.xlsx')
df3 = pd.read_excel(f'{DATA_PATH}{file_lst[1]}.xlsx')
df5 = pd.read_excel(f'{DATA_PATH}{file_lst[2]}.xlsx')
# df7 = pd.read_excel(f'{DATA_PATH}{file_lst[3]}.xlsx')

In [160]:
df['암호화요양기호'].nunique(), df3['암호화요양기호'].nunique(), df5['암호화요양기호'].nunique()
# , df7['암호화요양기호'].nunique()

(78126, 103188, 75025)

In [161]:
# 1. '약국' 삭제
mask = df3['종별코드명'] == '약국'
phar_idx = df3.loc[mask, :].index
print(len(phar_idx))

df3.drop(phar_idx, axis=0, inplace=True)


mask = df['종별코드명'] == '약국'
phar_idx = df.loc[mask, :].index
print(len(phar_idx))

df.drop(phar_idx, axis=0, inplace=True)

25062
0


In [162]:
df.shape, df3.shape

((78126, 30), (78126, 30))

In [163]:
merge_df = pd.merge(df3, df, on="암호화요양기호", how='left')

In [164]:
merge_df.shape

(78126, 59)

In [165]:
df5.head()

,암호화요양기호,요양기관명,진료과목코드,진료과목코드명,과목별 전문의수,선택진료 의사수
0,JDQ4MTg4MSM1MSMkMSMkMCMkODkkMzgxMzUxIzExIyQxIy...,가톨릭대학교 여의도성모병원,1,내과,37,0
1,JDQ4MTg4MSM1MSMkMSMkMCMkODkkMzgxMzUxIzExIyQxIy...,중앙대학교병원,1,내과,60,0
2,JDQ4MTg4MSM1MSMkMSMkMCMkODkkMzgxMzUxIzExIyQyIy...,학교법인가톨릭학원가톨릭대학교서울성모병원,1,내과,145,0
3,JDQ4MTg4MSM1MSMkMSMkMCMkODkkMzgxMzUxIzExIyQxIy...,서울대학교병원,1,내과,176,0
4,JDQ4MTg4MSM1MSMkMSMkMCMkODkkMzgxMzUxIzExIyQyIy...,한림대학교 강남성심병원,1,내과,39,0


In [166]:
agg_dict = {
    '과목별 전문의수' : sum,
}

pre_merge_df5 = df5.groupby(['암호화요양기호','진료과목코드명']).agg(agg_dict).reset_index()

/var/folders/67/63m6y1m97g90zkz5c8wkzrdr0000gn/T/ipykernel_79665/3282394653.py:5: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  pre_merge_df5 = df5.groupby(['암호화요양기호','진료과목코드명']).agg(agg_dict).reset_index()


In [167]:
merge_df = pd.merge(merge_df, pre_merge_df5, on="암호화요양기호", how='left')

In [168]:
# df7.head()

In [169]:
# pre_merge_df7 = df7.groupby(['암호화요양기호', '장비코드명'])['장비대수'].sum().reset_index()

In [170]:
# merge_df = pd.merge(merge_df, pre_merge_df7, on="요양기관명", how='left')

In [171]:
merge_df.shape

(407400, 61)

In [172]:
merge_df.columns

Index(['암호화요양기호', '요양기관명_x', '종별코드_x', '종별코드명_x', '설립구분코드', '설립구분코드명',
       '시도코드_x', '시도코드명_x', '시군구코드_x', '시군구코드명_x', '읍면동_x', '우편번호_x', '주소_x',
       '전화번호_x', '개설일자_x', '일반입원실상급병상수', '일반입원실일반병상수', '성인중환자병상수', '소아중환자병상수',
       '신생아중환자병상수', '분만실병상수', '수술실병상수', '응급실병상수', '물리치료실병상수', '정신과폐쇄상급병상수',
       '정신과폐쇄일반병상수', '정신과개방상급병상수', '정신과폐쇄개방병상수', '격리병실병상수', '무균치료실병상수',
       '요양기관명_y', '종별코드_y', '종별코드명_y', '시도코드_y', '시도코드명_y', '시군구코드_y',
       '시군구코드명_y', '읍면동_y', '우편번호_y', '주소_y', '전화번호_y', '병원홈페이지', '개설일자_y',
       '총의사수', '의과일반의 인원수', '의과인턴 인원수', '의과레지던트 인원수', '의과전문의 인원수', '치과일반의 인원수',
       '치과인턴 인원수', '치과레지던트 인원수', '치과전문의 인원수', '한방일반의 인원수', '한방인턴 인원수',
       '한방레지던트 인원수', '한방전문의 인원수', '조산사 인원수', '좌표(X)', '좌표(Y)', '진료과목코드명',
       '과목별 전문의수'],
      dtype='object')

In [173]:
# 합치기 '일반입원실상급병상수', '일반입원실일반병상수'

merge_df['일반입원실병상수'] = merge_df['일반입원실상급병상수'] + merge_df['일반입원실일반병상수']

In [174]:
no_use = ['종별코드_x', '설립구분코드', '시도코드_x', '시군구코드_x', '읍면동_x', '우편번호_x', '주소_x',
          '전화번호_x', '요양기관명_y', '종별코드_y', '종별코드명_y', '시도코드_y', '시도코드명_y', '시군구코드_y',
          '시군구코드명_y', '읍면동_y', '우편번호_y', '주소_y', '전화번호_y', '병원홈페이지', '개설일자_y', '좌표(X)', '좌표(Y)',
          '일반입원실상급병상수', '일반입원실일반병상수']



merge_df.drop(no_use, axis=1, inplace=True)

In [175]:
merge_df.head()

,암호화요양기호,요양기관명_x,종별코드명_x,설립구분코드명,시도코드명_x,시군구코드명_x,개설일자_x,성인중환자병상수,소아중환자병상수,신생아중환자병상수,...,치과레지던트 인원수,치과전문의 인원수,한방일반의 인원수,한방인턴 인원수,한방레지던트 인원수,한방전문의 인원수,조산사 인원수,진료과목코드명,과목별 전문의수,일반입원실병상수
0,JDQ4MTYyMiM4MSMkMiMkMCMkMDAkNDgxMTkxIzMxIyQxIy...,뽀빠이연합의원,의원,개인,대구,대구중구,20230424,0,0,0,...,0,0,0,0,0,0,0,내과,0.0,0
1,JDQ4MTYyMiM4MSMkMiMkMCMkMDAkNDgxMTkxIzMxIyQxIy...,뽀빠이연합의원,의원,개인,대구,대구중구,20230424,0,0,0,...,0,0,0,0,0,0,0,마취통증의학과,0.0,0
2,JDQ4MTYyMiM4MSMkMiMkMCMkMDAkNDgxMTkxIzMxIyQxIy...,뽀빠이연합의원,의원,개인,대구,대구중구,20230424,0,0,0,...,0,0,0,0,0,0,0,신경외과,0.0,0
3,JDQ4MTYyMiM4MSMkMiMkMCMkMDAkNDgxMTkxIzMxIyQxIy...,뽀빠이연합의원,의원,개인,대구,대구중구,20230424,0,0,0,...,0,0,0,0,0,0,0,외과,1.0,0
4,JDQ4MTYyMiM4MSMkMiMkMCMkMDAkNDgxMTkxIzMxIyQxIy...,뽀빠이연합의원,의원,개인,대구,대구중구,20230424,0,0,0,...,0,0,0,0,0,0,0,재활의학과,0.0,0


In [176]:
merge_df.columns

Index(['암호화요양기호', '요양기관명_x', '종별코드명_x', '설립구분코드명', '시도코드명_x', '시군구코드명_x',
       '개설일자_x', '성인중환자병상수', '소아중환자병상수', '신생아중환자병상수', '분만실병상수', '수술실병상수',
       '응급실병상수', '물리치료실병상수', '정신과폐쇄상급병상수', '정신과폐쇄일반병상수', '정신과개방상급병상수',
       '정신과폐쇄개방병상수', '격리병실병상수', '무균치료실병상수', '총의사수', '의과일반의 인원수', '의과인턴 인원수',
       '의과레지던트 인원수', '의과전문의 인원수', '치과일반의 인원수', '치과인턴 인원수', '치과레지던트 인원수',
       '치과전문의 인원수', '한방일반의 인원수', '한방인턴 인원수', '한방레지던트 인원수', '한방전문의 인원수',
       '조산사 인원수', '진료과목코드명', '과목별 전문의수', '일반입원실병상수'],
      dtype='object')

In [177]:
merge_df.columns = ['암호화요양기호', '요양기관명', '종별코드명', '설립구분코드명', '시도코드명', '시군구코드명',
       '개설일자', '성인중환자병상수', '소아중환자병상수', '신생아중환자병상수', '분만실병상수',
       '수술실병상수', '응급실병상수', '물리치료실병상수', '정신과폐쇄상급병상수', '정신과폐쇄일반병상수',
       '정신과개방상급병상수', '정신과폐쇄개방병상수', '격리병실병상수', '무균치료실병상수', '총의사수', '의과일반의 인원수',
       '의과인턴 인원수', '의과레지던트 인원수', '의과전문의 인원수', '치과일반의 인원수', '치과인턴 인원수',
       '치과레지던트 인원수', '치과전문의 인원수', '한방일반의 인원수', '한방인턴 인원수', '한방레지던트 인원수',
       '한방전문의 인원수', '조산사 인원수', '진료과목코드명', '과목별 전문의수', '일반입원실병상수']

In [178]:
# 일반입원실병상수 위치 옮기기
col1 = [merge_df.columns[-1]]
col2 = merge_df.columns[:8].to_list()
col3 = merge_df.columns[8:-1].to_list()
new_col = col2 + col1 + col3

merge_df = merge_df[new_col]
merge_df.columns

Index(['암호화요양기호', '요양기관명', '종별코드명', '설립구분코드명', '시도코드명', '시군구코드명', '개설일자',
       '성인중환자병상수', '일반입원실병상수', '소아중환자병상수', '신생아중환자병상수', '분만실병상수', '수술실병상수',
       '응급실병상수', '물리치료실병상수', '정신과폐쇄상급병상수', '정신과폐쇄일반병상수', '정신과개방상급병상수',
       '정신과폐쇄개방병상수', '격리병실병상수', '무균치료실병상수', '총의사수', '의과일반의 인원수', '의과인턴 인원수',
       '의과레지던트 인원수', '의과전문의 인원수', '치과일반의 인원수', '치과인턴 인원수', '치과레지던트 인원수',
       '치과전문의 인원수', '한방일반의 인원수', '한방인턴 인원수', '한방레지던트 인원수', '한방전문의 인원수',
       '조산사 인원수', '진료과목코드명', '과목별 전문의수'],
      dtype='object')

In [179]:
merge_df['개설일자'] = merge_df['개설일자'].astype('str')

In [180]:
merge_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 407400 entries, 0 to 407399
Data columns (total 37 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   암호화요양기호     407400 non-null  object 
 1   요양기관명       407400 non-null  object 
 2   종별코드명       407400 non-null  object 
 3   설립구분코드명     407400 non-null  object 
 4   시도코드명       407400 non-null  object 
 5   시군구코드명      407400 non-null  object 
 6   개설일자        407400 non-null  object 
 7   성인중환자병상수    407400 non-null  int64  
 8   일반입원실병상수    407400 non-null  int64  
 9   소아중환자병상수    407400 non-null  int64  
 10  신생아중환자병상수   407400 non-null  int64  
 11  분만실병상수      407400 non-null  int64  
 12  수술실병상수      407400 non-null  int64  
 13  응급실병상수      407400 non-null  int64  
 14  물리치료실병상수    407400 non-null  int64  
 15  정신과폐쇄상급병상수  407400 non-null  int64  
 16  정신과폐쇄일반병상수  407400 non-null  int64  
 17  정신과개방상급병상수  407400 non-null  int64  
 18  정신과폐쇄개방병상수  407400 non-null  int64  
 19  격리

In [181]:
merge_df.to_csv(f'/Users/hong/Desktop/practice_code/mid_project/data/병의원통합데이터/2024의원통합데이터.csv', encoding='UTF-8')

In [182]:
merge_df.columns

Index(['암호화요양기호', '요양기관명', '종별코드명', '설립구분코드명', '시도코드명', '시군구코드명', '개설일자',
       '성인중환자병상수', '일반입원실병상수', '소아중환자병상수', '신생아중환자병상수', '분만실병상수', '수술실병상수',
       '응급실병상수', '물리치료실병상수', '정신과폐쇄상급병상수', '정신과폐쇄일반병상수', '정신과개방상급병상수',
       '정신과폐쇄개방병상수', '격리병실병상수', '무균치료실병상수', '총의사수', '의과일반의 인원수', '의과인턴 인원수',
       '의과레지던트 인원수', '의과전문의 인원수', '치과일반의 인원수', '치과인턴 인원수', '치과레지던트 인원수',
       '치과전문의 인원수', '한방일반의 인원수', '한방인턴 인원수', '한방레지던트 인원수', '한방전문의 인원수',
       '조산사 인원수', '진료과목코드명', '과목별 전문의수'],
      dtype='object')

## 2023년

In [183]:
DATA_PATH = '/Users/hong/Desktop/practice_code/mid_project/data/전국병의원및약국현황/전국 병의원 및 약국 현황 2023.12/'

file_lst = ['1.병원정보서비스', '3.의료기관별상세정보서비스_01_시설정보', '5.의료기관별상세정보서비스_03_진료과목정보']

In [184]:
df = pd.read_excel(f'{DATA_PATH}{file_lst[0]}.xlsx')
df3 = pd.read_excel(f'{DATA_PATH}{file_lst[1]}.xlsx')
df5 = pd.read_excel(f'{DATA_PATH}{file_lst[2]}.xlsx')
# df7 = pd.read_excel(f'{DATA_PATH}{file_lst[3]}.xlsx')

In [185]:
df['암호화요양기호'].nunique(), df3['암호화요양기호'].nunique(), df5['암호화요양기호'].nunique()
# , df7['암호화요양기호'].nunique()

(77139, 101883, 74064)

In [186]:
# 1. '약국' 삭제
mask = df3['종별코드명'] == '약국'
phar_idx = df3.loc[mask, :].index
print(len(phar_idx))

df3.drop(phar_idx, axis=0, inplace=True)


mask = df['종별코드명'] == '약국'
phar_idx = df.loc[mask, :].index
print(len(phar_idx))

df.drop(phar_idx, axis=0, inplace=True)

24744
0


In [187]:
df.shape, df3.shape

((77139, 30), (77139, 28))

In [188]:
merge_df = pd.merge(df3, df, on="암호화요양기호", how='left')

In [189]:
merge_df.shape

(77139, 57)

In [190]:
agg_dict = {
    '과목별 전문의수' : sum,
}

pre_merge_df5 = df5.groupby(['암호화요양기호','진료과목코드명']).agg(agg_dict).reset_index()

/var/folders/67/63m6y1m97g90zkz5c8wkzrdr0000gn/T/ipykernel_79665/3282394653.py:5: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  pre_merge_df5 = df5.groupby(['암호화요양기호','진료과목코드명']).agg(agg_dict).reset_index()


In [191]:
merge_df = pd.merge(merge_df, pre_merge_df5, on="암호화요양기호", how='left')

In [192]:
# pre_merge_df7 = df7.groupby(['암호화요양기호', '장비코드명'])['장비대수'].sum().reset_index()
# # merge_df = pd.merge(merge_df, pre_merge_df7, on="요양기관명", how='left')

In [193]:
merge_df.shape

(378192, 59)

In [194]:
merge_df.columns

Index(['암호화요양기호', '요양기관명_x', '종별코드_x', '종별코드명_x', '설립구분코드', '설립구분코드명',
       '시도코드_x', '시도코드명_x', '시군구코드_x', '시군구코드명_x', '읍면동_x', '우편번호_x', '주소_x',
       '전화번호_x', '개설일자_x', '일반입원실상급병상수', '일반입원실일반병상수', '성인중환자병상수', '소아중환자병상수',
       '신생아중환자병상수', '분만실병상수', '수술실병상수', '응급실병상수', '물리치료실병상수', '정신과폐쇄상급병상수',
       '정신과폐쇄일반병상수', '격리병실병상수', '무균치료실병상수', '요양기관명_y', '종별코드_y', '종별코드명_y',
       '시도코드_y', '시도코드명_y', '시군구코드_y', '시군구코드명_y', '읍면동_y', '우편번호_y', '주소_y',
       '전화번호_y', '병원홈페이지', '개설일자_y', '총의사수', '의과일반의 인원수', '의과인턴 인원수',
       '의과레지던트 인원수', '의과전문의 인원수', '치과일반의 인원수', '치과인턴 인원수', '치과레지던트 인원수',
       '치과전문의 인원수', '한방일반의 인원수', '한방인턴 인원수', '한방레지던트 인원수', '한방전문의 인원수',
       '조산사 인원수', '좌표(X)', '좌표(Y)', '진료과목코드명', '과목별 전문의수'],
      dtype='object')

In [195]:
# 합치기 '일반입원실상급병상수', '일반입원실일반병상수'

merge_df['일반입원실병상수'] = merge_df['일반입원실상급병상수'] + merge_df['일반입원실일반병상수']

In [196]:
no_use = ['종별코드_x', '설립구분코드', '시도코드_x', '시군구코드_x', '읍면동_x', '우편번호_x', '주소_x',
          '전화번호_x', '요양기관명_y', '종별코드_y', '종별코드명_y', '시도코드_y', '시도코드명_y', '시군구코드_y',
          '시군구코드명_y', '읍면동_y', '우편번호_y', '주소_y', '전화번호_y', '병원홈페이지', '개설일자_y', '좌표(X)', '좌표(Y)',
          '일반입원실상급병상수', '일반입원실일반병상수']



merge_df.drop(no_use, axis=1, inplace=True)

In [197]:
merge_df.head()

,암호화요양기호,요양기관명_x,종별코드명_x,설립구분코드명,시도코드명_x,시군구코드명_x,개설일자_x,성인중환자병상수,소아중환자병상수,신생아중환자병상수,...,치과레지던트 인원수,치과전문의 인원수,한방일반의 인원수,한방인턴 인원수,한방레지던트 인원수,한방전문의 인원수,조산사 인원수,진료과목코드명,과목별 전문의수,일반입원실병상수
0,JDQ4MTg4MSM1MSMkMSMkMCMkNzIkNDgxNzAyIzExIyQxIy...,서울스타일치과의원,치과의원,개인,서울,중구,20080602.0,0,0,0,...,0,1,0,0,0,0,0,치과,0.0,0
1,JDQ4MTg4MSM1MSMkMSMkMCMkNzIkNDgxNzAyIzExIyQxIy...,서울닥터우치과의원,치과의원,개인,서울,중랑구,20200303.0,0,0,0,...,0,1,0,0,0,0,0,구강내과,0.0,0
2,JDQ4MTg4MSM1MSMkMSMkMCMkNzIkNDgxNzAyIzExIyQxIy...,서울닥터우치과의원,치과의원,개인,서울,중랑구,20200303.0,0,0,0,...,0,1,0,0,0,0,0,구강악안면외과,0.0,0
3,JDQ4MTg4MSM1MSMkMSMkMCMkNzIkNDgxNzAyIzExIyQxIy...,서울닥터우치과의원,치과의원,개인,서울,중랑구,20200303.0,0,0,0,...,0,1,0,0,0,0,0,소아치과,0.0,0
4,JDQ4MTg4MSM1MSMkMSMkMCMkNzIkNDgxNzAyIzExIyQxIy...,서울닥터우치과의원,치과의원,개인,서울,중랑구,20200303.0,0,0,0,...,0,1,0,0,0,0,0,영상치의학과,0.0,0


In [198]:
merge_df.columns

Index(['암호화요양기호', '요양기관명_x', '종별코드명_x', '설립구분코드명', '시도코드명_x', '시군구코드명_x',
       '개설일자_x', '성인중환자병상수', '소아중환자병상수', '신생아중환자병상수', '분만실병상수', '수술실병상수',
       '응급실병상수', '물리치료실병상수', '정신과폐쇄상급병상수', '정신과폐쇄일반병상수', '격리병실병상수', '무균치료실병상수',
       '총의사수', '의과일반의 인원수', '의과인턴 인원수', '의과레지던트 인원수', '의과전문의 인원수', '치과일반의 인원수',
       '치과인턴 인원수', '치과레지던트 인원수', '치과전문의 인원수', '한방일반의 인원수', '한방인턴 인원수',
       '한방레지던트 인원수', '한방전문의 인원수', '조산사 인원수', '진료과목코드명', '과목별 전문의수',
       '일반입원실병상수'],
      dtype='object')

In [199]:
merge_df.columns = ['암호화요양기호', '요양기관명', '종별코드명', '설립구분코드명', '시도코드명', '시군구코드명',
       '개설일자', '성인중환자병상수', '소아중환자병상수', '신생아중환자병상수', '분만실병상수',
       '수술실병상수', '응급실병상수', '물리치료실병상수', '정신과폐쇄상급병상수', '정신과폐쇄일반병상수', '격리병실병상수',
       '무균치료실병상수', '총의사수', '의과일반의 인원수', '의과인턴 인원수', '의과레지던트 인원수', '의과전문의 인원수',
       '치과일반의 인원수', '치과인턴 인원수', '치과레지던트 인원수', '치과전문의 인원수', '한방일반의 인원수',
       '한방인턴 인원수', '한방레지던트 인원수', '한방전문의 인원수', '조산사 인원수', '진료과목코드명', '과목별 전문의수',
       '일반입원실병상수']

In [200]:
# 일반입원실병상수 위치 옮기기
col1 = [merge_df.columns[-1]]
col2 = merge_df.columns[:8].to_list()
col3 = merge_df.columns[8:-1].to_list()
new_col = col2 + col1 + col3

merge_df = merge_df[new_col]
merge_df.columns

Index(['암호화요양기호', '요양기관명', '종별코드명', '설립구분코드명', '시도코드명', '시군구코드명', '개설일자',
       '성인중환자병상수', '일반입원실병상수', '소아중환자병상수', '신생아중환자병상수', '분만실병상수', '수술실병상수',
       '응급실병상수', '물리치료실병상수', '정신과폐쇄상급병상수', '정신과폐쇄일반병상수', '격리병실병상수', '무균치료실병상수',
       '총의사수', '의과일반의 인원수', '의과인턴 인원수', '의과레지던트 인원수', '의과전문의 인원수', '치과일반의 인원수',
       '치과인턴 인원수', '치과레지던트 인원수', '치과전문의 인원수', '한방일반의 인원수', '한방인턴 인원수',
       '한방레지던트 인원수', '한방전문의 인원수', '조산사 인원수', '진료과목코드명', '과목별 전문의수'],
      dtype='object')

In [201]:
merge_df['개설일자'] = merge_df['개설일자'].astype('str')

In [202]:
merge_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 378192 entries, 0 to 378191
Data columns (total 35 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   암호화요양기호     378192 non-null  object 
 1   요양기관명       378192 non-null  object 
 2   종별코드명       378192 non-null  object 
 3   설립구분코드명     378192 non-null  object 
 4   시도코드명       378192 non-null  object 
 5   시군구코드명      378192 non-null  object 
 6   개설일자        378192 non-null  object 
 7   성인중환자병상수    378192 non-null  int64  
 8   일반입원실병상수    378192 non-null  int64  
 9   소아중환자병상수    378192 non-null  int64  
 10  신생아중환자병상수   378192 non-null  int64  
 11  분만실병상수      378192 non-null  int64  
 12  수술실병상수      378192 non-null  int64  
 13  응급실병상수      378192 non-null  int64  
 14  물리치료실병상수    378192 non-null  int64  
 15  정신과폐쇄상급병상수  378192 non-null  int64  
 16  정신과폐쇄일반병상수  378192 non-null  int64  
 17  격리병실병상수     378192 non-null  int64  
 18  무균치료실병상수    378192 non-null  int64  
 19  총의

In [203]:
merge_df.to_csv(f'/Users/hong/Desktop/practice_code/mid_project/data/병의원통합데이터/2023의원통합데이터.csv', encoding='UTF-8')

In [204]:
merge_df.columns

Index(['암호화요양기호', '요양기관명', '종별코드명', '설립구분코드명', '시도코드명', '시군구코드명', '개설일자',
       '성인중환자병상수', '일반입원실병상수', '소아중환자병상수', '신생아중환자병상수', '분만실병상수', '수술실병상수',
       '응급실병상수', '물리치료실병상수', '정신과폐쇄상급병상수', '정신과폐쇄일반병상수', '격리병실병상수', '무균치료실병상수',
       '총의사수', '의과일반의 인원수', '의과인턴 인원수', '의과레지던트 인원수', '의과전문의 인원수', '치과일반의 인원수',
       '치과인턴 인원수', '치과레지던트 인원수', '치과전문의 인원수', '한방일반의 인원수', '한방인턴 인원수',
       '한방레지던트 인원수', '한방전문의 인원수', '조산사 인원수', '진료과목코드명', '과목별 전문의수'],
      dtype='object')

## 2022년

In [205]:
DATA_PATH = '/Users/hong/Desktop/practice_code/mid_project/data/전국병의원및약국현황/전국 병의원 및 약국 현황 2022.12/'

file_lst = ['1.병원정보서비스 2022.12', '3.의료기관별상세정보서비스_01_시설정보_2022.12', '5.의료기관별상세정보서비스_03_진료과목정보_2022.12']

In [206]:
df = pd.read_excel(f'{DATA_PATH}{file_lst[0]}.xlsx')
df3 = pd.read_excel(f'{DATA_PATH}{file_lst[1]}.xlsx')
df5 = pd.read_excel(f'{DATA_PATH}{file_lst[2]}.xlsx')
# df7 = pd.read_excel(f'{DATA_PATH}{file_lst[3]}.xlsx')

In [207]:
df['암호화요양기호'].nunique(), df3['암호화요양기호'].nunique(), df5['암호화요양기호'].nunique()
# , df7['암호화요양기호'].nunique()

(76249, 100638, 73200)

In [208]:
# 1. '약국' 삭제
mask = df3['종별코드명'] == '약국'
phar_idx = df3.loc[mask, :].index
print(len(phar_idx))

df3.drop(phar_idx, axis=0, inplace=True)


mask = df['종별코드명'] == '약국'
phar_idx = df.loc[mask, :].index
print(len(phar_idx))

df.drop(phar_idx, axis=0, inplace=True)

24389
0


In [209]:
df.shape, df3.shape

((76249, 30), (76249, 29))

In [210]:
merge_df = pd.merge(df3, df, on="암호화요양기호", how='left')

In [211]:
merge_df.shape

(76249, 58)

In [212]:
agg_dict = {
    '과목별 전문의수' : sum,
}

pre_merge_df5 = df5.groupby(['암호화요양기호','진료과목코드명']).agg(agg_dict).reset_index()

/var/folders/67/63m6y1m97g90zkz5c8wkzrdr0000gn/T/ipykernel_79665/3282394653.py:5: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  pre_merge_df5 = df5.groupby(['암호화요양기호','진료과목코드명']).agg(agg_dict).reset_index()


In [213]:
merge_df = pd.merge(merge_df, pre_merge_df5, on="암호화요양기호", how='left')

In [214]:
# pre_merge_df7 = df7.groupby(['암호화요양기호', '장비코드명'])['장비대수'].sum().reset_index()
# # merge_df = pd.merge(merge_df, pre_merge_df7, on="요양기관명", how='left')

In [215]:
merge_df.shape

(371191, 60)

In [216]:
merge_df.columns

Index(['암호화요양기호', '요양기관명_x', '종별코드_x', '종별코드명_x', '설립구분코드', '설립구분코드명',
       '시도코드_x', '시도코드명_x', '시군구코드_x', '시군구코드명_x', '읍면동_x', '우편번호_x', '주소_x',
       '전화번호_x', '병원링크', '개설일자_x', '일반입원실상급병상수', '일반입원실일반병상수', '성인중환자병상수',
       '소아중환자병상수', '신생아중환자병상수', '분만실병상수', '수술실병상수', '응급실병상수', '물리치료실병상수',
       '정신과폐쇄상급병상수', '정신과폐쇄일반병상수', '격리병실병상수', '무균치료실병상수', '요양기관명_y', '종별코드_y',
       '종별코드명_y', '시도코드_y', '시도코드명_y', '시군구코드_y', '시군구코드명_y', '읍면동_y',
       '우편번호_y', '주소_y', '전화번호_y', '병원홈페이지', '개설일자_y', '총의사수', '의과일반의 인원수',
       '의과인턴 인원수', '의과레지던트 인원수', '의과전문의 인원수', '치과일반의 인원수', '치과인턴 인원수',
       '치과레지던트 인원수', '치과전문의 인원수', '한방일반의 인원수', '한방인턴 인원수', '한방레지던트 인원수',
       '한방전문의 인원수', '조산사 인원수', '좌표(X)', '좌표(Y)', '진료과목코드명', '과목별 전문의수'],
      dtype='object')

In [217]:
# 합치기 '일반입원실상급병상수', '일반입원실일반병상수'

merge_df['일반입원실병상수'] = merge_df['일반입원실상급병상수'] + merge_df['일반입원실일반병상수']

In [218]:
no_use = ['종별코드_x', '설립구분코드', '시도코드_x', '시군구코드_x', '읍면동_x', '우편번호_x', '주소_x',
          '전화번호_x', '병원링크', '요양기관명_y', '종별코드_y', '종별코드명_y', '시도코드_y', '시도코드명_y', '시군구코드_y',
          '시군구코드명_y', '읍면동_y', '우편번호_y', '주소_y', '전화번호_y', '병원홈페이지', '개설일자_y', '좌표(X)', '좌표(Y)',
          '일반입원실상급병상수', '일반입원실일반병상수']



merge_df.drop(no_use, axis=1, inplace=True)

In [219]:
merge_df.head()

,암호화요양기호,요양기관명_x,종별코드명_x,설립구분코드명,시도코드명_x,시군구코드명_x,개설일자_x,성인중환자병상수,소아중환자병상수,신생아중환자병상수,...,치과레지던트 인원수,치과전문의 인원수,한방일반의 인원수,한방인턴 인원수,한방레지던트 인원수,한방전문의 인원수,조산사 인원수,진료과목코드명,과목별 전문의수,일반입원실병상수
0,JDU4MTI3MSM1MSMkMiMkMCMkMDAkMzgxNzAyIzExIyQxIy...,서울주니어소아청소년과의원,의원,개인,인천,인천부평구,20100813.0,0,0,0,...,0,0,0,0,0,0,0,소아청소년과,1.0,0
1,JDU4MTI3MSM1MSMkMiMkMCMkMDAkMzgxNzAyIzExIyQxIy...,서울주니어소아청소년과의원,의원,개인,인천,인천부평구,20100813.0,0,0,0,...,0,0,0,0,0,0,0,이비인후과,0.0,0
2,JDQ4MTYyMiM1MSMkMSMkMCMkMTMkNDgxMTkxIzMxIyQxIy...,동원당한의원,한의원,개인,경기,고양일산동구,20081103.0,0,0,0,...,0,0,1,0,0,0,0,침구과,0.0,0
3,JDQ4MTYyMiM1MSMkMSMkMCMkMTMkNDgxMTkxIzMxIyQxIy...,동원당한의원,한의원,개인,경기,고양일산동구,20081103.0,0,0,0,...,0,0,1,0,0,0,0,한방내과,0.0,0
4,JDQ4MTYyMiM1MSMkMSMkMCMkMTMkNDgxMTkxIzMxIyQxIy...,동원당한의원,한의원,개인,경기,고양일산동구,20081103.0,0,0,0,...,0,0,1,0,0,0,0,한방부인과,0.0,0


In [220]:
merge_df.columns

Index(['암호화요양기호', '요양기관명_x', '종별코드명_x', '설립구분코드명', '시도코드명_x', '시군구코드명_x',
       '개설일자_x', '성인중환자병상수', '소아중환자병상수', '신생아중환자병상수', '분만실병상수', '수술실병상수',
       '응급실병상수', '물리치료실병상수', '정신과폐쇄상급병상수', '정신과폐쇄일반병상수', '격리병실병상수', '무균치료실병상수',
       '총의사수', '의과일반의 인원수', '의과인턴 인원수', '의과레지던트 인원수', '의과전문의 인원수', '치과일반의 인원수',
       '치과인턴 인원수', '치과레지던트 인원수', '치과전문의 인원수', '한방일반의 인원수', '한방인턴 인원수',
       '한방레지던트 인원수', '한방전문의 인원수', '조산사 인원수', '진료과목코드명', '과목별 전문의수',
       '일반입원실병상수'],
      dtype='object')

In [221]:
merge_df.columns = ['암호화요양기호', '요양기관명', '종별코드명', '설립구분코드명', '시도코드명', '시군구코드명',
       '개설일자', '성인중환자병상수', '소아중환자병상수', '신생아중환자병상수', '분만실병상수',
       '수술실병상수', '응급실병상수', '물리치료실병상수', '정신과폐쇄상급병상수', '정신과폐쇄일반병상수', '격리병실병상수',
       '무균치료실병상수', '총의사수', '의과일반의 인원수', '의과인턴 인원수', '의과레지던트 인원수', '의과전문의 인원수',
       '치과일반의 인원수', '치과인턴 인원수', '치과레지던트 인원수', '치과전문의 인원수', '한방일반의 인원수',
       '한방인턴 인원수', '한방레지던트 인원수', '한방전문의 인원수', '조산사 인원수', '진료과목코드명', '과목별 전문의수',
       '일반입원실병상수']

In [222]:
# 일반입원실병상수 위치 옮기기
col1 = [merge_df.columns[-1]]
col2 = merge_df.columns[:8].to_list()
col3 = merge_df.columns[8:-1].to_list()
new_col = col2 + col1 + col3

merge_df = merge_df[new_col]
merge_df.columns

Index(['암호화요양기호', '요양기관명', '종별코드명', '설립구분코드명', '시도코드명', '시군구코드명', '개설일자',
       '성인중환자병상수', '일반입원실병상수', '소아중환자병상수', '신생아중환자병상수', '분만실병상수', '수술실병상수',
       '응급실병상수', '물리치료실병상수', '정신과폐쇄상급병상수', '정신과폐쇄일반병상수', '격리병실병상수', '무균치료실병상수',
       '총의사수', '의과일반의 인원수', '의과인턴 인원수', '의과레지던트 인원수', '의과전문의 인원수', '치과일반의 인원수',
       '치과인턴 인원수', '치과레지던트 인원수', '치과전문의 인원수', '한방일반의 인원수', '한방인턴 인원수',
       '한방레지던트 인원수', '한방전문의 인원수', '조산사 인원수', '진료과목코드명', '과목별 전문의수'],
      dtype='object')

In [223]:
merge_df['개설일자'] = merge_df['개설일자'].astype('str')

In [224]:
merge_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 371191 entries, 0 to 371190
Data columns (total 35 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   암호화요양기호     371191 non-null  object 
 1   요양기관명       371191 non-null  object 
 2   종별코드명       371191 non-null  object 
 3   설립구분코드명     371191 non-null  object 
 4   시도코드명       371191 non-null  object 
 5   시군구코드명      371191 non-null  object 
 6   개설일자        371191 non-null  object 
 7   성인중환자병상수    371191 non-null  int64  
 8   일반입원실병상수    371191 non-null  int64  
 9   소아중환자병상수    371191 non-null  int64  
 10  신생아중환자병상수   371191 non-null  int64  
 11  분만실병상수      371191 non-null  int64  
 12  수술실병상수      371191 non-null  int64  
 13  응급실병상수      371191 non-null  int64  
 14  물리치료실병상수    371191 non-null  int64  
 15  정신과폐쇄상급병상수  371191 non-null  int64  
 16  정신과폐쇄일반병상수  371191 non-null  int64  
 17  격리병실병상수     371191 non-null  int64  
 18  무균치료실병상수    371191 non-null  int64  
 19  총의

In [225]:
merge_df.to_csv(f'/Users/hong/Desktop/practice_code/mid_project/data/병의원통합데이터/2022의원통합데이터.csv', encoding='UTF-8')

In [226]:
merge_df.columns

Index(['암호화요양기호', '요양기관명', '종별코드명', '설립구분코드명', '시도코드명', '시군구코드명', '개설일자',
       '성인중환자병상수', '일반입원실병상수', '소아중환자병상수', '신생아중환자병상수', '분만실병상수', '수술실병상수',
       '응급실병상수', '물리치료실병상수', '정신과폐쇄상급병상수', '정신과폐쇄일반병상수', '격리병실병상수', '무균치료실병상수',
       '총의사수', '의과일반의 인원수', '의과인턴 인원수', '의과레지던트 인원수', '의과전문의 인원수', '치과일반의 인원수',
       '치과인턴 인원수', '치과레지던트 인원수', '치과전문의 인원수', '한방일반의 인원수', '한방인턴 인원수',
       '한방레지던트 인원수', '한방전문의 인원수', '조산사 인원수', '진료과목코드명', '과목별 전문의수'],
      dtype='object')

## 2021년

In [227]:
DATA_PATH = '/Users/hong/Desktop/practice_code/mid_project/data/전국병의원및약국현황/건강보험심사평가원_전국 병의원 및 약국 현황 2021.12/'

file_lst = ['1. 병원정보서비스 2021.12', '3. 의료기관별상세정보서비스 01시설정보 2021.12', '5. 의료기관별상세정보서비스 03진료과목정보 2021.12']

In [228]:
df = pd.read_excel(f'{DATA_PATH}{file_lst[0]}.xlsx')
df3 = pd.read_excel(f'{DATA_PATH}{file_lst[1]}.xlsx')
df5 = pd.read_excel(f'{DATA_PATH}{file_lst[2]}.xlsx')
# df7 = pd.read_excel(f'{DATA_PATH}{file_lst[3]}.xlsx')

In [229]:
df['암호화요양기호'].nunique(), df3['암호화요양기호'].nunique(), df5['암호화요양기호'].nunique()
# , df7['암호화요양기호'].nunique()

(74756, 98596, 71700)

In [230]:
# 1. '약국' 삭제
mask = df3['종별코드명'] == '약국'
phar_idx = df3.loc[mask, :].index
print(len(phar_idx))

df3.drop(phar_idx, axis=0, inplace=True)


mask = df['종별코드명'] == '약국'
phar_idx = df.loc[mask, :].index
print(len(phar_idx))

df.drop(phar_idx, axis=0, inplace=True)

23840
0


In [231]:
df.shape, df3.shape

((74756, 29), (74756, 29))

In [232]:
merge_df = pd.merge(df3, df, on="암호화요양기호", how='left')

In [233]:
merge_df.shape

(74756, 57)

In [234]:
agg_dict = {
    '과목별 전문의수' : sum,
}

pre_merge_df5 = df5.groupby(['암호화요양기호','진료과목코드명']).agg(agg_dict).reset_index()

/var/folders/67/63m6y1m97g90zkz5c8wkzrdr0000gn/T/ipykernel_79665/3282394653.py:5: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  pre_merge_df5 = df5.groupby(['암호화요양기호','진료과목코드명']).agg(agg_dict).reset_index()


In [235]:
merge_df = pd.merge(merge_df, pre_merge_df5, on="암호화요양기호", how='left')

In [236]:
# pre_merge_df7 = df7.groupby(['암호화요양기호', '장비코드명'])['장비대수'].sum().reset_index()
# # merge_df = pd.merge(merge_df, pre_merge_df7, on="요양기관명", how='left')

In [237]:
merge_df.shape

(358932, 59)

In [238]:
merge_df.columns

Index(['암호화요양기호', '요양기관명_x', '종별코드_x', '종별코드명_x', '설립구분코드', '설립구분코드명',
       '시도코드_x', '시도코드명_x', '시군구코드_x', '시군구코드명_x', '읍면동_x', '우편번호_x', '주소_x',
       '전화번호_x', '병원URL_x', '개설일자_x', '일반입원실상급병상수', '일반입원실일반병상수', '성인중환자병상수',
       '소아중환자병상수', '신생아중환자병상수', '정신과폐쇄상급병상수', '정신과폐쇄일반병상수', '격리병실병상수',
       '무균치료실병상수', '분만실병상수', '수술실병상수', '응급실병상수', '물리치료실병상수', '요양기관명_y',
       '종별코드_y', '종별코드명_y', '시도코드_y', '시도코드명_y', '시군구코드_y', '시군구코드명_y',
       '읍면동_y', '우편번호_y', '주소_y', '전화번호_y', '병원URL_y', '개설일자_y', '총의사수',
       '의과일반의 인원수', '의과인턴 인원수', '의과레지던트 인원수', '의과전문의 인원수', '치과일반의 인원수',
       '치과인턴 인원수', '치과레지던트 인원수', '치과전문의 인원수', '한방일반의 인원수', '한방인턴 인원수',
       '한방레지던트 인원수', '한방전문의 인원수', 'x좌표', 'y좌표', '진료과목코드명', '과목별 전문의수'],
      dtype='object')

In [239]:
# 합치기 '일반입원실상급병상수', '일반입원실일반병상수'

merge_df['일반입원실병상수'] = merge_df['일반입원실상급병상수'] + merge_df['일반입원실일반병상수']

In [240]:
no_use = ['종별코드_x', '설립구분코드', '시도코드_x', '시군구코드_x', '읍면동_x', '우편번호_x', '주소_x',
          '전화번호_x', '병원URL_x', '요양기관명_y', '종별코드_y', '종별코드명_y', '시도코드_y', '시도코드명_y', '시군구코드_y',
          '시군구코드명_y', '읍면동_y', '우편번호_y', '주소_y', '전화번호_y', '병원URL_y', '개설일자_y', 'x좌표', 'y좌표',
          '일반입원실상급병상수', '일반입원실일반병상수']



merge_df.drop(no_use, axis=1, inplace=True)

In [241]:
merge_df.head()

,암호화요양기호,요양기관명_x,종별코드명_x,설립구분코드명,시도코드명_x,시군구코드명_x,개설일자_x,성인중환자병상수,소아중환자병상수,신생아중환자병상수,...,치과인턴 인원수,치과레지던트 인원수,치과전문의 인원수,한방일반의 인원수,한방인턴 인원수,한방레지던트 인원수,한방전문의 인원수,진료과목코드명,과목별 전문의수,일반입원실병상수
0,JDQ4MTg4MSM1MSMkMSMkMCMkODkkMzgxMzUxIzExIyQxIy...,서울대학교병원,상급종합,공립,서울,종로구,1981-07-09,122,24,43,...,0,0,1,0,0,0,0,가정의학과,26.0,1508
1,JDQ4MTg4MSM1MSMkMSMkMCMkODkkMzgxMzUxIzExIyQxIy...,서울대학교병원,상급종합,공립,서울,종로구,1981-07-09,122,24,43,...,0,0,1,0,0,0,0,내과,145.0,1508
2,JDQ4MTg4MSM1MSMkMSMkMCMkODkkMzgxMzUxIzExIyQxIy...,서울대학교병원,상급종합,공립,서울,종로구,1981-07-09,122,24,43,...,0,0,1,0,0,0,0,마취통증의학과,42.0,1508
3,JDQ4MTg4MSM1MSMkMSMkMCMkODkkMzgxMzUxIzExIyQxIy...,서울대학교병원,상급종합,공립,서울,종로구,1981-07-09,122,24,43,...,0,0,1,0,0,0,0,방사선종양학과,8.0,1508
4,JDQ4MTg4MSM1MSMkMSMkMCMkODkkMzgxMzUxIzExIyQxIy...,서울대학교병원,상급종합,공립,서울,종로구,1981-07-09,122,24,43,...,0,0,1,0,0,0,0,병리과,20.0,1508


In [242]:
merge_df.columns

Index(['암호화요양기호', '요양기관명_x', '종별코드명_x', '설립구분코드명', '시도코드명_x', '시군구코드명_x',
       '개설일자_x', '성인중환자병상수', '소아중환자병상수', '신생아중환자병상수', '정신과폐쇄상급병상수',
       '정신과폐쇄일반병상수', '격리병실병상수', '무균치료실병상수', '분만실병상수', '수술실병상수', '응급실병상수',
       '물리치료실병상수', '총의사수', '의과일반의 인원수', '의과인턴 인원수', '의과레지던트 인원수', '의과전문의 인원수',
       '치과일반의 인원수', '치과인턴 인원수', '치과레지던트 인원수', '치과전문의 인원수', '한방일반의 인원수',
       '한방인턴 인원수', '한방레지던트 인원수', '한방전문의 인원수', '진료과목코드명', '과목별 전문의수',
       '일반입원실병상수'],
      dtype='object')

In [243]:
merge_df.columns = ['암호화요양기호', '요양기관명', '종별코드명', '설립구분코드명', '시도코드명', '시군구코드명',
       '개설일자', '성인중환자병상수', '소아중환자병상수', '신생아중환자병상수', '정신과폐쇄상급병상수',
       '정신과폐쇄일반병상수', '격리병실병상수', '무균치료실병상수', '분만실병상수', '수술실병상수', '응급실병상수',
       '물리치료실병상수', '총의사수', '의과일반의 인원수', '의과인턴 인원수', '의과레지던트 인원수', '의과전문의 인원수',
       '치과일반의 인원수', '치과인턴 인원수', '치과레지던트 인원수', '치과전문의 인원수', '한방일반의 인원수',
       '한방인턴 인원수', '한방레지던트 인원수', '한방전문의 인원수', '진료과목코드명', '과목별 전문의수',
       '일반입원실병상수']

In [244]:
# 일반입원실병상수 위치 옮기기
col1 = [merge_df.columns[-1]]
col2 = merge_df.columns[:8].to_list()
col3 = merge_df.columns[8:-1].to_list()
new_col = col2 + col1 + col3

merge_df = merge_df[new_col]
merge_df.columns

Index(['암호화요양기호', '요양기관명', '종별코드명', '설립구분코드명', '시도코드명', '시군구코드명', '개설일자',
       '성인중환자병상수', '일반입원실병상수', '소아중환자병상수', '신생아중환자병상수', '정신과폐쇄상급병상수',
       '정신과폐쇄일반병상수', '격리병실병상수', '무균치료실병상수', '분만실병상수', '수술실병상수', '응급실병상수',
       '물리치료실병상수', '총의사수', '의과일반의 인원수', '의과인턴 인원수', '의과레지던트 인원수', '의과전문의 인원수',
       '치과일반의 인원수', '치과인턴 인원수', '치과레지던트 인원수', '치과전문의 인원수', '한방일반의 인원수',
       '한방인턴 인원수', '한방레지던트 인원수', '한방전문의 인원수', '진료과목코드명', '과목별 전문의수'],
      dtype='object')

In [245]:
merge_df['개설일자'] = merge_df['개설일자'].astype('str')

In [246]:
merge_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 358932 entries, 0 to 358931
Data columns (total 34 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   암호화요양기호     358932 non-null  object 
 1   요양기관명       358932 non-null  object 
 2   종별코드명       358932 non-null  object 
 3   설립구분코드명     358932 non-null  object 
 4   시도코드명       358932 non-null  object 
 5   시군구코드명      358932 non-null  object 
 6   개설일자        358932 non-null  object 
 7   성인중환자병상수    358932 non-null  int64  
 8   일반입원실병상수    358932 non-null  int64  
 9   소아중환자병상수    358932 non-null  int64  
 10  신생아중환자병상수   358932 non-null  int64  
 11  정신과폐쇄상급병상수  358932 non-null  int64  
 12  정신과폐쇄일반병상수  358932 non-null  int64  
 13  격리병실병상수     358932 non-null  int64  
 14  무균치료실병상수    358932 non-null  int64  
 15  분만실병상수      358932 non-null  int64  
 16  수술실병상수      358932 non-null  int64  
 17  응급실병상수      358932 non-null  int64  
 18  물리치료실병상수    358932 non-null  int64  
 19  총의

In [247]:
merge_df.to_csv(f'/Users/hong/Desktop/practice_code/mid_project/data/병의원통합데이터/2021의원통합데이터.csv', encoding='UTF-8')

In [248]:
merge_df.columns

Index(['암호화요양기호', '요양기관명', '종별코드명', '설립구분코드명', '시도코드명', '시군구코드명', '개설일자',
       '성인중환자병상수', '일반입원실병상수', '소아중환자병상수', '신생아중환자병상수', '정신과폐쇄상급병상수',
       '정신과폐쇄일반병상수', '격리병실병상수', '무균치료실병상수', '분만실병상수', '수술실병상수', '응급실병상수',
       '물리치료실병상수', '총의사수', '의과일반의 인원수', '의과인턴 인원수', '의과레지던트 인원수', '의과전문의 인원수',
       '치과일반의 인원수', '치과인턴 인원수', '치과레지던트 인원수', '치과전문의 인원수', '한방일반의 인원수',
       '한방인턴 인원수', '한방레지던트 인원수', '한방전문의 인원수', '진료과목코드명', '과목별 전문의수'],
      dtype='object')

In [249]:
merge_df.head()

,암호화요양기호,요양기관명,종별코드명,설립구분코드명,시도코드명,시군구코드명,개설일자,성인중환자병상수,일반입원실병상수,소아중환자병상수,...,치과일반의 인원수,치과인턴 인원수,치과레지던트 인원수,치과전문의 인원수,한방일반의 인원수,한방인턴 인원수,한방레지던트 인원수,한방전문의 인원수,진료과목코드명,과목별 전문의수
0,JDQ4MTg4MSM1MSMkMSMkMCMkODkkMzgxMzUxIzExIyQxIy...,서울대학교병원,상급종합,공립,서울,종로구,1981-07-09,122,1508,24,...,1,0,0,1,0,0,0,0,가정의학과,26.0
1,JDQ4MTg4MSM1MSMkMSMkMCMkODkkMzgxMzUxIzExIyQxIy...,서울대학교병원,상급종합,공립,서울,종로구,1981-07-09,122,1508,24,...,1,0,0,1,0,0,0,0,내과,145.0
2,JDQ4MTg4MSM1MSMkMSMkMCMkODkkMzgxMzUxIzExIyQxIy...,서울대학교병원,상급종합,공립,서울,종로구,1981-07-09,122,1508,24,...,1,0,0,1,0,0,0,0,마취통증의학과,42.0
3,JDQ4MTg4MSM1MSMkMSMkMCMkODkkMzgxMzUxIzExIyQxIy...,서울대학교병원,상급종합,공립,서울,종로구,1981-07-09,122,1508,24,...,1,0,0,1,0,0,0,0,방사선종양학과,8.0
4,JDQ4MTg4MSM1MSMkMSMkMCMkODkkMzgxMzUxIzExIyQxIy...,서울대학교병원,상급종합,공립,서울,종로구,1981-07-09,122,1508,24,...,1,0,0,1,0,0,0,0,병리과,20.0
